In [1]:
from mycelia.shared.checkpoint import start_model_from
from mycelia.shared.config import MinerConfig
from mycelia.shared.app_logging import structlog, configure_logging

miner_config_path = "/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/config.yaml"
configure_logging()
config = MinerConfig.from_path(miner_config_path)
a, b, c = start_model_from(0, config)

2025-11-25 07:46:47 [info     ] found existing config path /home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/config.yaml [mycelia.shared.config]
2025-11-25 07:46:47 [info     ] rank 0: Latest checkpoint found in /home/isabella/crucible/subnet-MoE/checkpoints/miner/validator_checkpoint/uid_16_hotkey_5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH_globalver_4_block_5771091.pt [mycelia.shared.checkpoint]
2025-11-25 07:46:47 [info     ] rank 0: Latest checkpoint found in /home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/inneropt_80 [mycelia.shared.checkpoint]
2025-11-25 07:46:47 [info     ] validator checkpoint version 4 > miner checkpoint version (-1, 80) [mycelia.shared.checkpoint]


In [3]:
from mycelia.shared.checkpoint import *
rank = 0

validator_ckpt_found, validator_version, latest_validator_ckpt = get_resume_info(rank, config, config.miner.validator_checkpoint_path)
miner_ckpt_found, miner_version, latest_miner_ckpt = get_resume_info(rank, config, config.ckpt.checkpoint_path)

2025-11-25 07:46:58 [info     ] rank 0: Latest checkpoint found in /home/isabella/crucible/subnet-MoE/checkpoints/miner/validator_checkpoint/uid_16_hotkey_5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH_globalver_4_block_5771091.pt [mycelia.shared.checkpoint]
2025-11-25 07:46:58 [info     ] rank 0: Latest checkpoint found in /home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/inneropt_80 [mycelia.shared.checkpoint]


In [4]:
validator_version

{'uid': 16,
 'hotkey': '5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH',
 'globalver': 4,
 'block': 5771091,
 'filename': PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/validator_checkpoint/uid_16_hotkey_5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH_globalver_4_block_5771091.pt'),
 'inneropt': -1}

In [5]:
miner_version

{'inneropt': 80,
 'filename': PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/inneropt_80'),
 'globalver': -1}

## signing with hotkey

In [2]:
import bittensor as bt
from mycelia.shared.schema import * 
# 1. Load your wallet & hotkey
#    Assumes you've already created a wallet with this name
wallet = bt.wallet(
    name="miner",   # your wallet name
    hotkey="hk1"  # your hotkey name
)

wallet2 = bt.wallet(
    name="miner",   # your wallet name
    hotkey="hk2"  # your hotkey name
)

target_ss58 = wallet2.hotkey.ss58_address
block = 123
message = construct_block_message(target_hotkey_ss58=target_ss58, block=block)
signature = sign_message(wallet.hotkey, message)

verify_message(wallet.hotkey.ss58_address, message, signature_hex=signature)

True

In [10]:
signature2 = hotkey.sign(b'24')

In [15]:
from substrateinterface import SubstrateInterface, Keypair

# --- What Bob receives ---
signature_hex = signature.hex()   # Alice's signature in hex
hotkey_ss58 = wallet.hotkey.ss58_address     # Alice's hotkey address

# --- Convert hex → bytes ---
signature = bytes.fromhex(signature_hex.replace("0x", ""))

# --- Construct a Keypair from the SS58 address ---
keypair = Keypair(ss58_address=hotkey_ss58)

# --- Verify signature ---
is_valid = keypair.verify(message, signature)

print("Valid signature?", is_valid)


Valid signature? True


In [3]:
import bittensor as bt 

sub = bt.subtensor()
meta = sub.metagraph(netuid = 9)


metagraph(netuid:9, n:256, block:6959063, network:finney)

In [ ]:
meta.axons[2].hotkey

'5EUdJNUX9XUT5mNcfg8Y1nfsBqnnUDpDMk2wxQnQsDS6U2Cb'

## save statedict per expert group 

In [1]:
from mycelia.shared.config import ValidatorConfig
from mycelia.shared.checkpoint import *
from mycelia.shared.app_logging import configure_logging
from mycelia.shared.expert_manager import ExpertManager
from mycelia.shared.model import load_model 

rank = 0

miner_config_path = "/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/config.yaml"
configure_logging()
config = ValidatorConfig.from_path(miner_config_path)

logger.info(f"rank {rank} setup training - load model and expert manager")
expert_manager = ExpertManager(config)
base_model, version = load_model(rank, config, expert_manager)

/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-11-27 05:52:56 [info     ] found existing config path /home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/config.yaml [mycelia.shared.config]
2025-11-27 05:52:56 [info     ] rank 0 setup training - load model and expert manager [mycelia.shared.checkpoint]
2025-11-27 05:52:56 [info     ] loading task folder          [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_dummy'),)
2025-11-27 05:52:56 [info     ] loading task folder          [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)
2025-11-27 05:52:56 [warning  ] model.chain_lookup_failed    [mycelia.shared.model] error=exceptions must derive from BaseException
2025-11-27 05:52:56 [info     ] returning regular validator checkpoint [mycelia.shared.checkpoint]
2025-11-27 05:52:56 [info     ] rank 0: No checkpoints found in /home/isabella/crucible/subnet-MoE/checkpoints/

The fast path is not available because one of the required library is not installed. Falling back to torch implementation. To install follow https://github.com/fla-org/flash-linear-attention#installation and https://github.com/Dao-AILab/causal-conv1d


In [3]:
from mycelia.shared.checkpoint import * 
from mycelia.shared.expert_manager import get_layer_expert_id
global_opt_step = -1
ckpt_path = os.path.join(config.ckpt.checkpoint_path, f"globalopt_{int(global_opt_step)}")
split_state_dict_by_expert_group(base_model.state_dict(), expert_manager.expert_group_assignment, get_layer_expert_id, save_dir = ckpt_path)

{1: '/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/globalopt_-1/group_1.pt',
 0: '/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/globalopt_-1/group_0.pt',
 'shared': '/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/globalopt_-1/shared.pt'}

In [6]:
resume, start_step, latest_checkpoint_path = get_resume_info(rank=0, config=config)
ckpt_dir = latest_checkpoint_path

2025-11-27 07:09:58 [info     ] rank 0: Latest checkpoint found in /home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/globalopt_-1 [mycelia.shared.checkpoint]


In [10]:
import glob

expert_group_id = 1
group_pattern = os.path.join(ckpt_dir, f"group_{expert_group_id}*.pt")
group_files = glob.glob(group_pattern)
group_files

['/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/globalopt_-1/group_1.pt']